In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_15m.parquet")
df_label_side = pd.read_parquet("data/label_15m_side.parquet")

df_features = df_features.iloc[240:]
df_label_side = df_label_side.iloc[240:]

print(df_features.shape)
print(df_label_side.shape)

df_label_side.head()

(178032, 1263)
(178032, 8)


,open,high,low,close,volume,ret,trgt,bin
index,,,,,,,,
2020-01-03 12:00:00+00:00,7338.40,7347.00,7320.01,7344.13,1485.629,NaN,NaN,0.0
2020-01-03 12:15:00+00:00,7344.84,7348.52,7335.10,7336.76,1387.831,NaN,NaN,0.0
2020-01-03 12:30:00+00:00,7337.05,7337.06,7315.39,7317.80,1354.678,NaN,NaN,0.0
2020-01-03 12:45:00+00:00,7317.82,7323.67,7306.01,7321.57,1362.688,NaN,NaN,0.0
2020-01-03 13:00:00+00:00,7321.31,7329.00,7310.85,7324.98,783.078,NaN,NaN,0.0


In [3]:
from custom_indicators.selection import SIDE_ALL

valid_mask = (df_label_side["bin"].astype(int) != 0).tolist()
side_features = df_features[valid_mask][SIDE_ALL]
side_label = (df_label_side[valid_mask]["bin"].astype(int) == 1).astype(int)
print(side_label.value_counts())

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features.iloc[:train_test_split_point]
side_features_test = side_features.iloc[train_test_split_point:]
side_label_train = side_label.iloc[:train_test_split_point]
side_label_test = side_label.iloc[train_test_split_point:]

side_label_test.value_counts()

bin
1    10427
0    10157
Name: count, dtype: int64


bin
1    2076
0    2041
Name: count, dtype: int64

In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 100,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=50, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_82788/2690221176.py:41: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2025-02-24 22:07:48,996] A new study created in memory with name: no-name-32e79e13-feed-42d3-a041-896b2e8867ff
[I 2025-02-24 22:07:55,197] Trial 0 finished with value: 0.6826700047862744 and parameters: {'num_leaves': 245, 'max_depth': 54, 'min_gain_to_split': 0.36985648123091536, 'min_data_in_leaf': 155, 'lambda_l1': 0.5033598133739919, 'lambda_l2': 0.09125418421821364, 'num_boost_round': 541}. Best is trial 0 with value: 0.6826

precision: 0.6694267515923567, recall: 0.50626204238921, f1: 0.5765222161272627


[I 2025-02-24 22:08:01,313] Trial 1 finished with value: 0.6830511602703349 and parameters: {'num_leaves': 70, 'max_depth': 67, 'min_gain_to_split': 0.42619246639701935, 'min_data_in_leaf': 77, 'lambda_l1': 0.44950345382201606, 'lambda_l2': 0.5323917774159685, 'num_boost_round': 876}. Best is trial 1 with value: 0.6830511602703349.


precision: 0.6597938144329897, recall: 0.4932562620423892, f1: 0.5644983461962514


[I 2025-02-24 22:08:08,042] Trial 2 finished with value: 0.6854645943136795 and parameters: {'num_leaves': 133, 'max_depth': 43, 'min_gain_to_split': 0.35067837054033263, 'min_data_in_leaf': 84, 'lambda_l1': 0.2033801720255551, 'lambda_l2': 0.6199748120929661, 'num_boost_round': 887}. Best is trial 2 with value: 0.6854645943136795.


...

precision: 0.6635279347143754, recall: 0.5091522157996147, f1: 0.5761787953120742


[I 2025-02-24 22:11:47,826] Trial 48 finished with value: 0.6851057417356523 and parameters: {'num_leaves': 71, 'max_depth': 76, 'min_gain_to_split': 0.7365268372871164, 'min_data_in_leaf': 176, 'lambda_l1': 0.5513145488532778, 'lambda_l2': 0.8685216866340054, 'num_boost_round': 998}. Best is trial 37 with value: 0.7009130738927138.


precision: 0.6377708978328174, recall: 0.5953757225433526, f1: 0.6158445440956651


[I 2025-02-24 22:11:51,480] Trial 49 finished with value: 0.6777928666574151 and parameters: {'num_leaves': 136, 'max_depth': 95, 'min_gain_to_split': 0.30760941392472074, 'min_data_in_leaf': 194, 'lambda_l1': 0.7760910144660571, 'lambda_l2': 0.042658072393528146, 'num_boost_round': 641}. Best is trial 37 with value: 0.7009130738927138.


precision: 0.6439732142857143, recall: 0.5558766859344894, f1: 0.5966907962771458


In [6]:
study.best_params

{'num_leaves': 60,
 'max_depth': 58,
 'min_gain_to_split': 0.48176635792830663,
 'min_data_in_leaf': 191,
 'lambda_l1': 0.4842426606577862,
 'lambda_l2': 0.8401075140310387,
 'num_boost_round': 812}

In [7]:
dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(study.best_params, dtrain)

In [9]:
import plotly.express as px

res = model.predict(df_features[SIDE_ALL][df_label_side["bin"].astype(int) != 0])

fig = px.histogram(res, nbins=100)
fig.show()

In [10]:
import plotly.express as px

res = model.predict(df_features[SIDE_ALL][df_label_side["bin"].astype(int) == 0])

fig = px.histogram(res, nbins=100)
fig.show()

In [11]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.selection import SIDE_ALL

df_features = pd.read_parquet("data/features_15m.parquet")
df_label = pd.read_parquet("data/label_15m_meta.parquet")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
model_side_res = model_side.predict(df_features[SIDE_ALL])
df_features["model_side_res"] = model_side_res

df_features = df_features[df_label["ret"].notna()].iloc[240:]
df_label = df_label[df_label["ret"].notna()].iloc[240:]

print(df_features.shape)

assert df_features.shape[0] == df_label.shape[0]
print(df_label.shape)
df_label.head()

(34977, 1264)
(34977, 9)


,open,high,low,close,volume,ret,trgt,bin,side
index,,,,,,,,,
2020-01-14 02:45:00+00:00,8421.35,8440.00,8410.00,8438.64,1784.328,0.009365,0.005059,0.0,1.0
2020-01-14 04:00:00+00:00,8469.54,8598.27,8469.40,8548.31,8772.286,-0.002025,0.005723,0.0,1.0
2020-01-14 04:15:00+00:00,8548.94,8581.45,8515.80,8532.90,3475.529,-0.000105,0.005871,0.0,1.0
2020-01-14 04:30:00+00:00,8533.92,8554.85,8525.00,8547.34,1797.736,-0.002497,0.005882,0.0,1.0
2020-01-14 09:15:00+00:00,8529.99,8550.00,8522.69,8550.00,1916.405,0.000583,0.002437,0.0,-1.0


In [2]:
from custom_indicators.selection import META_ALL

# valid_mask = df_label["ret"].notna().tolist()
valid_mask = [True] * df_features.shape[0]

meta_features = df_features[valid_mask][META_ALL]
print(meta_features.shape)
meta_label = df_label[valid_mask]["bin"].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features.iloc[:train_test_split_point]
meta_features_test = meta_features.iloc[train_test_split_point:]
meta_label_train = meta_label.iloc[:train_test_split_point]
meta_label_test = meta_label.iloc[train_test_split_point:]

meta_label_test.value_counts()

(34977, 1027)


bin
0    4287
1    2709
Name: count, dtype: int64

In [3]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 100,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 300, 1000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}")
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=50, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_97831/1983182041.py:42: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2025-02-24 22:33:02,692] A new study created in memory with name: no-name-a75245b0-375a-4e77-8e37-cde8d8fd4ba2
[I 2025-02-24 22:33:12,387] Trial 0 finished with value: 0.4785714285714286 and parameters: {'num_leaves': 83, 'max_depth': 55, 'min_gain_to_split': 0.08476357935115539, 'min_data_in_leaf': 29, 'lambda_l1': 0.6458443074124002, 'lambda_l2': 0.9961686382023126, 'num_boost_round': 361}. Best is trial 0 with value: 0.4785714

precision: 0.5173745173745173, recall: 0.44518272425249167, f1: 0.4785714285714286, auc: 0.618831749269362


[I 2025-02-24 22:33:19,954] Trial 1 finished with value: 0.48114129372679304 and parameters: {'num_leaves': 96, 'max_depth': 46, 'min_gain_to_split': 0.82380446174861, 'min_data_in_leaf': 142, 'lambda_l1': 0.45150273354693926, 'lambda_l2': 0.0809410949918626, 'num_boost_round': 389}. Best is trial 1 with value: 0.48114129372679304.


precision: 0.5112126245847176, recall: 0.4544112218530823, f1: 0.48114129372679304, auc: 0.6208832440707064


[I 2025-02-24 22:33:28,431] Trial 2 finished with value: 0.4852855193919314 and parameters: {'num_leaves': 45, 'max_depth': 77, 'min_gain_to_split': 0.6801512619003673, 'min_data_in_leaf': 29, 'lambda_l1': 0.06569805133012406, 'lambda_l2': 0.056980666564593684, 'num_boost_round': 493}. Best is trial 2 with value: 0.4852855193919314.


...

precision: 0.5089285714285714, recall: 0.4629014396456257, f1: 0.4848250531606418, auc: 0.6154406046833667


[I 2025-02-24 22:40:20,160] Trial 48 finished with value: 0.4833036949219522 and parameters: {'num_leaves': 63, 'max_depth': 59, 'min_gain_to_split': 0.46566966360674733, 'min_data_in_leaf': 32, 'lambda_l1': 0.5680023813737746, 'lambda_l2': 0.35982088344591023, 'num_boost_round': 850}. Best is trial 28 with value: 0.49275921992662675.


precision: 0.5199829931972789, recall: 0.4514581026208933, f1: 0.4833036949219522, auc: 0.6211783321162135


[I 2025-02-24 22:40:29,353] Trial 49 finished with value: 0.4866217516843118 and parameters: {'num_leaves': 156, 'max_depth': 53, 'min_gain_to_split': 0.6973169280001594, 'min_data_in_leaf': 74, 'lambda_l1': 0.47670671391293745, 'lambda_l2': 0.1991752632557778, 'num_boost_round': 899}. Best is trial 28 with value: 0.49275921992662675.


precision: 0.5084473049074819, recall: 0.46659283868586193, f1: 0.4866217516843118, auc: 0.6176893701915265


In [5]:
study.best_params

{'num_leaves': 111,
 'max_depth': 52,
 'min_gain_to_split': 0.9195525272241841,
 'min_data_in_leaf': 58,
 'lambda_l1': 0.9027259499319511,
 'lambda_l2': 0.00040518311288572154,
 'num_boost_round': 693}

In [6]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    study.best_params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [8]:
import plotly.express as px

res = model.predict(df_features[META_ALL])

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
model.save_model("custom_indicators/models/model_meta.txt")